In [ ]:
# this library was built against GRCh37 / hg19

In [1]:
library(infercnv)

The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 



In [2]:
library(rtracklayer)

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb



In [3]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::%within%() masks IRanges::%within%()
✖ dplyr::collapse()     masks IRanges::collapse()
✖ dplyr::combine()      masks BiocGenerics::combine()
✖ dplyr::desc()         masks IRanges::desc()
✖ tidyr::expand()       masks S4Vectors::expand()
✖ dplyr::filter()       masks stats::filter()
✖ dplyr::first()        masks S4Vectors::first()
✖ dplyr::lag()          masks stats::lag()
✖ ggplot2::Position()   masks BiocGenerics::Position(), base::Position()
✖ purrr::reduce()       masks GenomicRanges::reduce(), IRanges::reduce()
✖ dplyr::rename()       masks S4Vectors::rename()
✖ lubridate::second()   masks S4Vectors::second()
✖ lubridate::second<-

# Prepare files for infercnv

### prepare gene position file (one time only, then read from disk)

#### I downloaded the refGene gtf file from https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/genes/

In [4]:
gtf <- rtracklayer::import('data/hg19.refGene.gtf')

In [5]:
gtf_df=as.data.frame(gtf)

In [6]:
head(gtf_df)

,seqnames,start,end,width,strand,source,type,score,phase,gene_id,transcript_id,gene_name,exon_number,exon_id
,<fct>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,chr1,11869,14362,2494,+,refGene,transcript,NA,NA,LOC102725121,NR_148357,LOC102725121,NA,NA
2,chr1,11869,12227,359,+,refGene,exon,NA,NA,LOC102725121,NR_148357,LOC102725121,1,NR_148357.1
3,chr1,12613,12721,109,+,refGene,exon,NA,NA,LOC102725121,NR_148357,LOC102725121,2,NR_148357.2
4,chr1,13221,14362,1142,+,refGene,exon,NA,NA,LOC102725121,NR_148357,LOC102725121,3,NR_148357.3
5,chr1,11874,14409,2536,+,refGene,transcript,NA,NA,DDX11L1,NR_046018,DDX11L1,NA,NA
6,chr1,11874,12227,354,+,refGene,exon,NA,NA,DDX11L1,NR_046018,DDX11L1,1,NR_046018.1


In [29]:
gtf_infercnv_input = gtf_df %>% filter(type=="transcript") %>% select("gene_id", "seqnames", "start", "end") %>% unique()

#### are genes still duplicated here? yes, annotated with slightly different start/stops

In [30]:
gtf_infercnv_input %>% nrow()

[1] 46562

In [31]:
gtf_infercnv_input %>% select("gene_id") %>% unique() %>% nrow()

[1] 28265

In [35]:
gtf_infercnv_input[gtf_infercnv_input %>% select("gene_id") %>% duplicated(),]

,gene_id,seqnames,start,end
,<chr>,<fct>,<int>,<int>
26,LINC01128,chr1,762971,794826
30,LINC01128,chr1,763178,794826
42,MAS1L,chr6_apd_hap1,757952,759211
43,MAS1L,chr6_cox_hap2,973255,974514
44,MAS1L,chr6_dbb_hap3,757805,759064
94,ACYP1,chr14,75519946,75530727
114,MORF4L2,chrX,102930426,102941746
132,MMP23A,chr1,1631378,1633247
148,PUF60,chr8,144898514,144911211


#### drop duplicate rows -- they're really similar so can just randomly pick one

In [40]:
gtf_infercnv_input = gtf_infercnv_input[!duplicated(gtf_infercnv_input$gene_id),]

#### filter to genes on the 23 chromosomes

In [47]:
gtf_infercnv_input = gtf_infercnv_input %>% filter(seqnames %in% paste0("chr", c(c(1:22), "X", "Y")))

In [49]:
write.table(gtf_infercnv_input, file="data/infercnv_input_files/sys/gene_pos.txt", col.names=F, row.names=F, quote=F, sep="\t")

#### prepared the other 2 input files in python, writing from anndata object to disk

# Run infercnv

In [4]:
infercnv_obj = CreateInfercnvObject(raw_counts_matrix="data/infercnv_input_files/sys/counts.txt",
                                    annotations_file="data/infercnv_input_files/sys/cell_annots.txt",
                                    delim="\t",
                                    gene_order_file="data/infercnv_input_files/sys/gene_pos.txt",
                                    ref_group_names=c("endothelial","macrophage","fibroblast","t.cell","t.cd4","t.cd8","nk","mastocyte","b.cell"))

INFO [2023-07-24 16:05:23] Parsing matrix: data/infercnv_input_files/sys/counts.txt
INFO [2023-07-24 16:06:22] Parsing gene order file: data/infercnv_input_files/sys/gene_pos.txt
INFO [2023-07-24 16:06:22] Parsing cell annotations file: data/infercnv_input_files/sys/cell_annots.txt
INFO [2023-07-24 16:06:22] ::order_reduce:Start.
INFO [2023-07-24 16:06:23] .order_reduce(): expr and order match.
INFO [2023-07-24 16:06:24] ::process_data:order_reduce:Reduction from positional data, new dimensions (r,c) = 23686,7109 Total=2333571590 Min=0 Max=202951.
INFO [2023-07-24 16:06:25] num genes removed taking into account provided gene ordering list: 3427 = 14.4684623828422% removed.
INFO [2023-07-24 16:06:25] -filtering out cells < 100 or > Inf, removing 0 % of cells
WARN [2023-07-24 16:06:26] Please use "options(scipen = 100)" before running infercnv if you are using the analysis_mode="subclusters" option or you may encounter an error while the hclust is being generated.
INFO [2023-07-24 16:06:

In [5]:
#if the instructions printed above say to run this, run this
options(scipen = 100)

In [8]:
# perform infercnv operations to reveal cnv signal
# perform infercnv operations to reveal cnv signal
infercnv_obj = infercnv::run(infercnv_obj,
                             cutoff=1,  # use 1 for smart-seq, 0.1 for 10x-genomics
                             out_dir="outputs/infercnv/sys_take2/",  # dir is auto-created for storing outputs
                             cluster_by_groups=T,   # cluster
                             denoise=T,
                             HMM=T,
                             analysis_mode='subclusters',
                             leiden_resolution=0.01, #initial run returned too many subclusters
                             debug=TRUE
                             )

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

Centering and scaling data matrix

PC_ 1 
Positive:  NSUN3, ARL6, CRYBG3, ARL13B, CPOX, CLDND1, PROS1, ST3GAL6, DCBLD2, EPHA3 
	   COL8A1, C3orf38, CMSS1, ZNF654, CGGBP1, FILIP1L, CHMP2B, GBE1, TBC1D23, ROBO1 
	   ROBO2, NIT2, ZNF717, LNP1, FAM86DP, PDZRN3, TMEM45A, EBLN2, TFG, PPP4R2 
Negative:  DEAF1, TMEM80, EPS8L2, IRF7, TALDO1, PHRF1, RASSF7, SLC25A22, RPLP2, LRRC56 
	   HRAS, PNPLA2, RNH1, CD151, PTDSS2, POLR2L, ANO9, SIGIRR, TSPAN4, PKP3 
	   CHID1, B4GALNT4, AP2A2, IFITM3, MUC6, MUC5B, IFITM1, TOLLIP, IFITM2, MOB2 
PC_ 2 
Positive:  CLSTN3, PEX5, CD163L1, C1RL-AS1, CD163, C1RL, NANOG, C1R, SLC2A3, C1S 
	   LPCAT3, FOXJ2, EMG1, PHB2, PTPN6, C3AR1, C12orf57, ATN1, 

INFO [2023-07-24 17:12:47] define_signif_tumor_subclusters(), tumor: malignant_SyS10


Centering and scaling data matrix

PC_ 1 
Positive:  ZNF526, DEDD2, CIC, ERF, POU2F2, PAFAH1B3, ZNF574, PRR19, RABAC1, ARHGEF1 
	   ZNF576, ZNF428, MEGF8, IRGQ, PLAUR, CD79A, PINLYP, RPS19, LIPE, SMG9 
	   XRCC1, BCKDHA, ETHE1, CEACAM1, LYPD3, PHLDB3, KCNN4, EXOSC5, ZNF283, TMEM91 
Negative:  FAIM, PIK3CB, CEP70, MRPS22, ESYT3, COPB2, RBP1, CLSTN2, SLC25A36, ZBTB38 
	   MRAS, RASA2, RNF7, ARMC8, ATP1B3, TFDP2, PCOLCE2, DBR1, TRPC1, PLS1 
	   GK5, ATR, XRN1, U2SURP, CHST2, DZIP1L, SLC9A9, PLOD2, NCK1, PLSCR4 
PC_ 2 
Positive:  CLEC5A, MTRNR2L6, SSBP1, AGK, EPHB6, CUL1, MRPS33, ARHGEF5, TPK1, EPHA1 
	   OR2A7, LOC154761, ZYX, ARHGEF35, EZH2, CASP2, CTAGE4, GSTK1, PDIA4, BRAF 
	   NDUFB2, ZNF786, ADCK2, ZNF425, MKRN1, SLC37A3, PARP12, TBXAS1, ZNF398, HIPK2 
Negative:  SPAG7, CAMTA2, KIF1C, ENO3, SLC52A1, PFN1, ZFP3, RNF167, ZNF232, SLC25A11 
	   ZNF594, C17orf107, SCIMP, CHRNE, RABEP1, NUP88, MINK1, RPAIN, PLD2, C1QBP 
	   ITGAE, CAMKK1, P2RX1, PSMB6, ATP2A3, DHX33, P2RX5, ZZEF1, EMC6, VM

INFO [2023-07-24 17:12:47] define_signif_tumor_subclusters(), tumor: malignant_SyS11


Centering and scaling data matrix

PC_ 1 
Positive:  ST3GAL6, ARHGAP10, TMEM184C, CPOX, EDNRA, NR3C2, SLC10A7, CLDND1, FILIP1L, DCLK2 
	   TBC1D23, LSM6, LRBA, ZNF827, RPS3A, NIT2, LNP1, SH3D19, MMAA, TMEM45A 
	   FAM160A1, TFG, SMAD1, FBXW7, ABI3BP, TMEM154, ARFIP1, OTUD4, ARL13B, TRIM2 
Negative:  SGTA, SLC39A3, THOP1, GNG7, GADD45B, ZNF554, LMNB2, ZNF555, TIMM13, ZNF57 
	   SPPL2B, ZNF77, LSM7, TLE2, GNA11, GNA15, OAZ1, S1PR4, NCLN, SF3A2 
	   NFIC, PLEKHJ1, DOHH, DOT1L, FZR1, AP3D1, MFSD12, MOB3A, HMG20B, MKNK2 
PC_ 2 
Positive:  GNG2, NID2, FRMD6, PTGDR, FRMD6-AS1, PTGER2, TMX1, TXNDC16, GPR137C, PYGL 
	   PSMC6, NIN, STYX, SAV1, GNPNAT1, ATL1, FERMT2, MAP4K5, DDHD1, L2HGDH 
	   BMP4, SOS2, CDKN3, ARF6, NEMF, KLHDC2, KLHDC1, POLE2, DNAAF2, MGAT2 
Negative:  PCDHB13, PCDHB12, PCDHB14, PCDHB11, PCDHB19P, PCDHB10, PCDHB15, PCDHB9, TAF7, PCDHGA2 
	   PCDHB16, PCDHGA3, PCDHGB1, PCDHB7, PCDHGA4, PCDHB6, PCDHGB2, PCDHGA5, PCDHB5, PCDHGB3 
	   PCDHGA6, PCDHB4, PCDHGA7, PCDHGA8, PCDHGB5, P

...

DEBUG [2023-07-24 19:30:03] inferCNV::heatmap.cnv x range set to:  -1,3
INFO [2023-07-24 19:30:09] Colors for breaks:  #00008B,#24249B,#4848AB,#6D6DBC,#9191CC,#B6B6DD,#DADAEE,#FFFFFF,#EEDADA,#DDB6B6,#CC9191,#BC6D6D,#AB4848,#9B2424,#8B0000
INFO [2023-07-24 19:30:09] Quantiles of plotted data range: 0.5,1,1,1,2
INFO [2023-07-24 19:30:12] 

	STEP 22: Denoising

INFO [2023-07-24 19:30:12] ::process_data:Remove noise, noise threshold defined via ref mean sd_amplifier:  1.5
INFO [2023-07-24 19:30:12] denoising using mean(normal) +- sd_amplifier * sd(normal) per gene per cell across all data
INFO [2023-07-24 19:30:13] :: **** clear_noise_via_ref_quantiles **** : removing noise between bounds:  0.906554811719053 - 1.09819581708522
INFO [2023-07-24 19:31:49] 

## Making the final infercnv heatmap ##
INFO [2023-07-24 19:31:50] ::plot_cnv:Start
INFO [2023-07-24 19:31:50] ::plot_cnv:Current data dimensions (r,c)=12073,6951 Total=84513229.2665191 Min=0.49605252441373 Max=2.22182072260546.
INFO [202

Warning message in (function (fmt, ...) :
“one argument not used by format 'inferCNV::heatmap.cnv x range set to:  0.72541708269308,1.27458291730692'”


DEBUG [2023-07-24 19:32:00] inferCNV::heatmap.cnv x range set to:  0.72541708269308,1.27458291730692
INFO [2023-07-24 19:32:09] Colors for breaks:  #00008B,#24249B,#4848AB,#6D6DBC,#9191CC,#B6B6DD,#DADAEE,#FFFFFF,#EEDADA,#DDB6B6,#CC9191,#BC6D6D,#AB4848,#9B2424,#8B0000
INFO [2023-07-24 19:32:09] Quantiles of plotted data range: 0.72541708269308,1.00237531440214,1.00237531440214,1.00237531440214,1.27458291730692
DEBUG [2023-07-24 19:32:15] ::inferCNV::heatmap.cnv colorkey z range: 0.72541708269308,0.764643213736925,0.803869344780771,0.843095475824617,0.882321606868463,0.921547737912308,0.960773868956154,1,1.03922613104385,1.07845226208769,1.11767839313154,1.15690452417538,1.19613065521923,1.23535678626307,1.27458291730692
DEBUG [2023-07-24 19:32:15] ::inferCNV::heatmap.cnv colorkey breaks range: 0.72541708269308,0.762028138334002,0.798639193974925,0.835250249615848,0.871861305256771,0.908472360897693,0.945083416538616,0.981694472179539,1.01830552782046,1.05491658346138,1.09152763910231,1.

Warning message in (function (fmt, ...) :
“one argument not used by format 'inferCNV::heatmap.cnv x range set to:  0.72541708269308,1.27458291730692'”


DEBUG [2023-07-24 19:33:14] inferCNV::heatmap.cnv x range set to:  0.72541708269308,1.27458291730692
INFO [2023-07-24 19:33:20] Colors for breaks:  #00008B,#24249B,#4848AB,#6D6DBC,#9191CC,#B6B6DD,#DADAEE,#FFFFFF,#EEDADA,#DDB6B6,#CC9191,#BC6D6D,#AB4848,#9B2424,#8B0000
INFO [2023-07-24 19:33:20] Quantiles of plotted data range: 0.72541708269308,1.00237531440214,1.00237531440214,1.00237531440214,1.27458291730692


In [6]:
infercnv_obj = readRDS('outputs/infercnv/sys_take2/run.final.infercnv_obj')